In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Activation, MaxPool2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [2]:
ds = xr.open_dataset("../../synthethic_holograms_v0.nc")

In [3]:
split_index = 7000
all_labels = ds[["x", "y", "z", "d"]].to_dataframe()
train_labels = all_labels.iloc[:split_index]
test_labels = all_labels.iloc[split_index:]
scaler = MinMaxScaler()
scaled_train_labels = pd.DataFrame(scaler.fit_transform(train_labels), index=train_labels.index, columns=train_labels.columns)
scaled_test_labels = pd.DataFrame(scaler.transform(test_labels), index=test_labels.index, columns=test_labels.columns)

In [4]:
in_data = ds["image"].transpose("hologram_number", "ysize", 'xsize').expand_dims("channel", 3)

In [5]:
scaled_in_data = in_data / 255

In [4]:
conv_input = Input(shape=(400, 600, 1))
conv_1 = Conv2D(8, (5, 5), padding="same")(conv_input)
act_1 = Activation("relu")(conv_1)
pool_1 = MaxPool2D(pool_size=(4, 4))(act_1)
conv_2 = Conv2D(16, (5, 5), padding="same")(pool_1)
act_2 = Activation("relu")(conv_2)
pool_2 = MaxPool2D(pool_size=(4, 4))(act_2)
conv_3 = Conv2D(32, (5, 5), padding="same")(pool_2)
act_3 = Activation("relu")(conv_3)
pool_3 = MaxPool2D(pool_size=(4, 4))(act_3)
flat = Flatten()(pool_3)
dense_1 = Dense(64, activation="relu")(flat)
dense_2 = Dense(32, activation="relu")(dense_1)
out = Dense(4)(dense_2)
mod = Model(conv_input, out)
mod.compile(optimizer="adam", loss="mae")

In [6]:
mod.fit(np.random.random(size=(400, 400, 600, 1)), np.random.random(size=(400, 4)), batch_size=64, epochs=2, verbose=1)

Train on 400 samples
Epoch 1/2
 64/400 [===>..........................] - ETA: 5s

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv2d/Conv2D (defined at /home/dgagne/miniconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_1066]

Function call stack:
distributed_function


In [7]:
mod.fit(scaled_in_data[:split_index].values, scaled_train_labels.values, batch_size=64, epochs=10, verbose=1)

Train on 7000 samples
Epoch 1/10
  64/7000 [..............................] - ETA: 1:48

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv2d/Conv2D (defined at /home/dgagne/miniconda3/envs/tf2/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_distributed_function_1066]

Function call stack:
distributed_function


In [7]:
! echo $LD_LIBRARY_PATH

/home/dgagne/miniconda3/envs/tf2/lib:/home/dgagne/miniconda3/envs/tf2/lib:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64


In [8]:
! ls /home/dgagne/miniconda3/envs/tf2/lib

charset.alias			  libQt5Charts.so.5.12.5
cmake				  libQt5Concurrent.prl
cudatoolkit_config.yaml		  libQt5Concurrent.so
dbus-1.0			  libQt5Concurrent.so.5
engines-1.1			  libQt5Concurrent.so.5.12
gettext				  libQt5Concurrent.so.5.12.5
girepository-1.0		  libQt5Core.prl
glib-2.0			  libQt5Core.so
gstreamer-1.0			  libQt5Core.so.5
icu				  libQt5Core.so.5.12
itcl4.1.2			  libQt5Core.so.5.12.5
krb5				  libQt5DataVisualization.prl
libasan.so			  libQt5DataVisualization.so
libasan.so.5			  libQt5DataVisualization.so.5
libasan.so.5.0.0		  libQt5DataVisualization.so.5.12
libatomic.so			  libQt5DataVisualization.so.5.12.5
libatomic.so.1			  libQt5DBus.prl
libatomic.so.1.2.0		  libQt5DBus.so
libblas.so			  libQt5DBus.so.5
libblas.so.3			  libQt5DBus.so.5.12
libbz2.a			  libQt5DBus.so.5.12.5
libbz2.so			  libQt5DesignerComponents.prl
libbz2.so.1.0			  libQt5DesignerComponents.so
libbz2.so.1.0.8			  libQt5DesignerComponents.so.5
libcblas.so			  libQt5DesignerComponents.so.5.12
libcblas.so.3			 